# gBizINFO API データ品質検証

このノートブックでは、経済産業省が提供するgBizINFO APIの**データ品質**を検証します。
具体的に以下の点を明らかにします：

1. **時間的整合性**: 公開日と実際のデータ更新日の乖離
2. **網羅性**: 全省庁のデータが適切に統合されているか
3. **年次分布**: 各年のデータ量の偏り（2015-2025年）
4. **欠損値**: 重要フィールドの欠損率

## 前提条件
- APIキーを取得済み
(以下の、経済産業省のAPI利用方法ページを参考にしてください。
https://info.gbiz.go.jp/api/index.html)

## 対象読者
- Pythonコードを理解できる方
- 生成AIツールを使用できる方（このプログラムをチャットの入力に使用してください）

In [ ]:
# 必要なライブラリのインストール
!pip install pandas numpy matplotlib requests tqdm

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import time
from tqdm.notebook import tqdm

# APIの基本設定
API_TOKEN = "YOUR_API_TOKEN"  # 実際のAPIトークンに置き換えてください
API_TOKEN = "Hgt7Gj3Il3rQovE8OiU817jgmrKe9jnM"  # テスト後に削除
BASE_URL = "https://info.gbiz.go.jp/hojin"
HEADERS = {"X-hojinInfo-api-token": API_TOKEN}

# APIリクエストの間隔（秒）- レート制限対策
REQUEST_INTERVAL = 0.5

print("API設定が完了しました")
print(f"Base URL: {BASE_URL}")

## 年ごとの件数の分布を取得
API利用中、各年で偏りを感じたので調査。

各年一様に分布していることを期待する。

- エンドポイント：
- 調査条件：過去のデータから現在までのデータ

### 開始年を調査

In [ ]:
{コード作って！紛失した。結果は、2015になるはず。}

### 開始年を調査

In [4]:
def get_yearly_update_count(start_year=2015, end_year=datetime.now().year):
    results = {}
    
    for year in range(start_year, end_year + 1):
        from_date = f"{year}0101"
        to_date = f"{year}1231"
        
        # 各情報カテゴリごとにリクエスト
        categories = [
            'updateInfo', 
            'updateInfo/certification', 
            'updateInfo/commendation', 
            'updateInfo/finance', 
            'updateInfo/patent', 
            'updateInfo/procurement', 
            'updateInfo/subsidy', 
            'updateInfo/workplace'
        ]
        
        total_count = 0
        for category in categories:
            url = f"{BASE_URL}/v1/hojin/{category}"
            
            params = {
                'from': from_date,
                'to': to_date,
                'page': 1,
                'limit': 1
            }
            
            try:
                response = requests.get(url, headers=HEADERS, params=params)
                
                if response.status_code == 200:
                    data = response.json()
                    total_count += int(data.get('totalCount', 0))
                    time.sleep(REQUEST_INTERVAL)
                
            except Exception as e:
                print(f"Error fetching data for {year} - {category}: {e}")
        
        results[year] = total_count
        print(f"{year}年: {total_count:,}件")
    
    return results

# 年次データ数を取得
yearly_counts = get_yearly_update_count()

# データフレームに変換して表示
df_yearly = pd.DataFrame.from_dict(yearly_counts, orient='index', columns=['更新件数'])
df_yearly.index.name = '年'
print(df_yearly)

2015年: 1,887,914件
2016年: 151,081件
2017年: 135,005件
2018年: 585,068件
2019年: 497,493件
2020年: 1,133,327件
2021年: 665,528件
2022年: 581,804件
2023年: 784,961件
2024年: 5,347,545件
2025年: 427,328件
         更新件数
年            
2015  1887914
2016   151081
2017   135005
2018   585068
2019   497493
2020  1133327
2021   665528
2022   581804
2023   784961
2024  5347545
2025   427328


In [ ]:
from matplotlib.ticker import FuncFormatter

# y軸の目盛りを見やすくするための設定
def format_yaxis(x, pos):
    return '{:,.0f}'.format(x)

# グラフ作成
plt.figure(figsize=(12, 6))
df_yearly['更新件数'].plot(kind='bar')
plt.title('gBizINFO 年次データ更新件数')
plt.xlabel('年')
plt.ylabel('更新件数')
plt.gca().yaxis.set_major_formatter(FuncFormatter(format_yaxis))
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 特定事例の分析

ニュースで報道された不正受給事例に関連する企業情報を分析します。

In [ ]:
def search_company(company_name):
    """企業情報をgBizINFO APIで検索する関数"""
    endpoint = f"{BASE_URL}/v1/hojin"
    params = {
        'name': company_name,
        'limit': '10'
    }
    
    try:
        response = requests.get(endpoint, headers=HEADERS, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"エラー: HTTPステータスコード {response.status_code}")
            print(f"レスポンス: {response.text}")
            return None
    except Exception as e:
        print(f"例外が発生しました: {e}")
        return None

def get_company_subsidy(corporate_number):
    """法人番号を使用して補助金情報を取得する関数"""
    endpoint = f"{BASE_URL}/v1/hojin/{corporate_number}"
    
    try:
        response = requests.get(endpoint, headers=HEADERS)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"エラー: HTTPステータスコード {response.status_code}")
            print(f"レスポンス: {response.text}")
            return None
    except Exception as e:
        print(f"例外が発生しました: {e}")
        return None

# ケース1: カネヨ山野辺水産
print(f"gBizINFO APIから「カネヨ山野辺水産」の情報を検索しています...")
data = search_company("カネヨ山野辺水産")
if data and "hojin-infos" in data and data["hojin-infos"]:
    print(f"検索結果: {len(data['hojin-infos'])}件")
    print("\n企業データ:")
    for company in data["hojin-infos"]:
        print(f"法人番号: {company.get('corporate_number')}")
        print(f"名称: {company.get('name')}")
        print(f"所在地: {company.get('location')}")
        print(f"ステータス: {company.get('status')}")
        print("---")
        
        # 補助金情報を確認
        corporate_number = company.get('corporate_number')
        if corporate_number:
            print("\n補助金情報を取得しています...")
            subsidy_data = get_company_subsidy(corporate_number)
            
            if subsidy_data and "hojin-infos" in subsidy_data and subsidy_data["hojin-infos"]:
                company_info = subsidy_data["hojin-infos"][0]
                if "subsidy" in company_info and company_info["subsidy"]:
                    print(f"補助金情報: {len(company_info['subsidy'])}件")
                    for subsidy in company_info["subsidy"]:
                        print(f"補助金名: {subsidy.get('title')}")
                        print(f"金額: {subsidy.get('amount')}円")
                        print(f"府省: {subsidy.get('government_departments')}")
                        print("---")
                else:
                    print("補助金情報はありません")
else:
    print("検索結果はありません")

In [ ]:
# ケース2: ビック・ママ
print(f"gBizINFO APIから「ビック・ママ」の情報を検索しています...")
data = search_company("ビック・ママ")
if data and "hojin-infos" in data and data["hojin-infos"]:
    print(f"検索結果: {len(data['hojin-infos'])}件")
    
    # 宮城県の対象企業を検索
    target_company = None
    for company in data["hojin-infos"]:
        if "宮城県" in company.get('location', ''):
            target_company = company
            break
    
    if target_company:
        print("\n対象企業の詳細:")
        print(f"法人番号: {target_company.get('corporate_number')}")
        print(f"名称: {target_company.get('name')}")
        print(f"所在地: {target_company.get('location')}")
        
        # 補助金情報を確認
        corporate_number = target_company.get('corporate_number')
        if corporate_number:
            print("\n補助金情報を取得しています...")
            subsidy_data = get_company_subsidy(corporate_number)
            
            if subsidy_data and "hojin-infos" in subsidy_data and subsidy_data["hojin-infos"]:
                company_info = subsidy_data["hojin-infos"][0]
                if "subsidy" in company_info and company_info["subsidy"]:
                    print(f"補助金情報: {len(company_info['subsidy'])}件")
                    for subsidy in company_info["subsidy"]:
                        print(f"補助金名: {subsidy.get('title')}")
                        print(f"金額: {subsidy.get('amount')}円")
                        print(f"府省: {subsidy.get('government_departments')}")
                        print("---")
                else:
                    print("補助金情報はありません")
    else:
        print("宮城県の企業が見つかりません")
else:
    print("検索結果はありません")

In [ ]:
# ケース3: ファイブエム商事
print(f"gBizINFO APIから「ファイブエム商事」の情報を検索しています...")
data = search_company("ファイブエム商事")
if data and "hojin-infos" in data and data["hojin-infos"]:
    print(f"検索結果: {len(data['hojin-infos'])}件")
    
    # 対象企業を検索
    target_company = data["hojin-infos"][0]  # 最初の結果を対象とする
    
    print("\n企業詳細:")
    print(f"法人番号: {target_company.get('corporate_number')}")
    print(f"名称: {target_company.get('name')}")
    print(f"所在地: {target_company.get('location')}")
    
    # 補助金情報を確認
    corporate_number = target_company.get('corporate_number')
    if corporate_number:
        print("\n補助金情報を取得しています...")
        subsidy_data = get_company_subsidy(corporate_number)
        
        if subsidy_data and "hojin-infos" in subsidy_data and subsidy_data["hojin-infos"]:
            company_info = subsidy_data["hojin-infos"][0]
            if "subsidy" in company_info and company_info["subsidy"]:
                print(f"補助金情報: {len(company_info['subsidy'])}件")
                for subsidy in company_info["subsidy"]:
                    print(f"補助金名: {subsidy.get('title')}")
                    print(f"金額: {subsidy.get('amount')}円")
                    print(f"認定日: {subsidy.get('date_of_approval')}")
                    print(f"府省: {subsidy.get('government_departments')}")
                    print("---")
            else:
                print("補助金情報はありません")
else:
    print("検索結果はありません")

# 統計的に異常な補助金受給パターンを持つ組織の分析

補助金受給に関して統計的に異常なパターンを示す組織を分析します。

In [ ]:
def analyze_organization(name, corporate_number):
    """特定の組織の補助金・調達パターンを分析する関数"""
    print(f"{name}（法人番号: {corporate_number}）を分析中...")
    
    # 組織の詳細を取得
    org_data = get_company_subsidy(corporate_number)
    
    if org_data and "hojin-infos" in org_data and org_data["hojin-infos"]:
        org_info = org_data["hojin-infos"][0]
        
        # 基本情報
        print(f"名称: {org_info.get('name')}")
        print(f"所在地: {org_info.get('location')}")
        print(f"代表者: {org_info.get('representative_name')}")
        
        # 補助金分析
        total_subsidy_amount = 0
        subsidy_count = 0
        if "subsidy" in org_info and org_info["subsidy"]:
            subsidies = org_info["subsidy"]
            subsidy_count = len(subsidies)
            
            # 総額を計算
            for subsidy in subsidies:
                amount_str = subsidy.get('amount')
                if amount_str and amount_str.isdigit():
                    total_subsidy_amount += int(amount_str)
            
            print(f"\n補助金情報: {subsidy_count}件")
            print(f"補助金総額: ¥{total_subsidy_amount:,}")
            
            # 最大の補助金を表示
            print("\n補助金額上位5件:")
            sorted_subsidies = sorted(subsidies, 
                                     key=lambda x: int(x.get('amount')) if x.get('amount') and x.get('amount').isdigit() else 0, 
                                     reverse=True)
            
            for i, subsidy in enumerate(sorted_subsidies[:5]):
                amount_str = subsidy.get('amount')
                amount = int(amount_str) if amount_str and amount_str.isdigit() else 0
                print(f"{i+1}. {subsidy.get('title')}")
                print(f"   金額: ¥{amount:,}")
                print(f"   府省: {subsidy.get('government_departments')}")
        else:
            print("補助金情報はありません")
        
        # 調達分析
        if "procurement" in org_info and org_info["procurement"]:
            procurements = org_info["procurement"]
            print(f"\n調達情報: {len(procurements)}件")
            
            # 総額を計算
            total_proc_amount = 0
            for proc in procurements:
                amount_str = proc.get('amount')
                if amount_str and amount_str.isdigit():
                    total_proc_amount += int(amount_str)
            
            print(f"調達総額: ¥{total_proc_amount:,}")
        else:
            print("\n調達情報はありません")
            
        return {
            "name": name,
            "subsidy_count": subsidy_count,
            "total_subsidy_amount": total_subsidy_amount
        }
    else:
        print("組織データの取得に失敗しました")
        return None

# 一般社団法人環境共創イニシアチブを分析
sii_results = analyze_organization(
    "一般社団法人環境共創イニシアチブ", 
    "8010605002161"
)

# ランドブレイン株式会社を分析
landbrain_results = analyze_organization(
    "ランドブレイン株式会社", 
    "9010001109037"
)